I'll use this notebook to study potential LJ constituents and look into whether I can unambiguously pick out the relevant particles with dR requirements. This study should be updated to instead use pfjet.pfcand and compare directly with ljsources.

In [4]:
# python
import sys
import os
import importlib
# columnar analysis
sys.path.append(os.path.join("~", "cms-jovyan")) # point to local coffea code
from coffea import processor

sys.path.append(os.path.join(os.getcwd()))
import awkward as ak
from dask.distributed import Client
# locala
sys.path.insert(1, os.path.join( os.getcwd(), '../..')) # fixme: there must be a better way to handle this... previously: 'sys.path[0]'
from sidm.tools import ffschema, sidm_processor, utilities
# always reload local modules to pick up changes during development
importlib.reload(ffschema)
importlib.reload(sidm_processor)
importlib.reload(utilities)
# plotting
import matplotlib.pyplot as plt
utilities.set_plot_style()

In [5]:
# samples = [
#     "2Mu2E_100GeV_5GeV_400mm",
#     "2Mu2E_100GeV_5GeV_200mm",
#     "2Mu2E_100GeV_5GeV_40mm",
#     "2Mu2E_100GeV_5GeV_4mm",
#     "2Mu2E_100GeV_5GeV_0p4mm",
# ]

samples = [
    "2Mu2E_800GeV_1p2GeV_1p2mm",
    "2Mu2E_800GeV_0p25GeV_1p2mm",
]

# samples = [
#     "2Mu2E_1000GeV_5GeV_40mm",
#     "2Mu2E_500GeV_5GeV_40mm",
#     "2Mu2E_100GeV_5GeV_40mm",
# ]

# samples = [
#     "2Mu2E_100GeV_5GeV_4mm",
#     "2Mu2E_150GeV_5GeV_2p7mm",
#     "2Mu2E_200GeV_5GeV_2mm",
#     "2Mu2E_500GeV_5GeV_0p8mm",
#     "2Mu2E_800GeV_5GeV_0p5mm",
#     "2Mu2E_1000GeV_5GeV_0p4mm",
# ]
fileset = utilities.make_fileset(samples, "ffntuple_v4")

In [6]:
runner = processor.Runner(
    #executor=processor.IterativeExecutor(),
    executor=processor.FuturesExecutor(),
    schema=ffschema.FFSchema,
)

p = sidm_processor.SidmProcessor(
    [
        "2mu2e",
        "2mu2e_matchedLj",
    ],
    ["base_plus_gen"]
)

output = runner.run(fileset, treename="ffNtuplizer/ffNtuple", processor_instance=p)
#hists = output["out"][samples[0]]["hists"]

ch1 = "2mu2e"
ch2 = "2mu2e_matchedLj"

Output()

concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/concurrent/futures/process.py", line 246, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
  File "/opt/conda/lib/python3.9/site-packages/coffea/processor/executor.py", line 1367, in automatic_retries
    raise e
  File "/opt/conda/lib/python3.9/site-packages/coffea/processor/executor.py", line 1336, in automatic_retries
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.9/site-packages/coffea/processor/executor.py", line 1417, in metadata_fetcher
    with uproot.open({item.filename: None}, timeout=xrootdtimeout) as file:
  File "/opt/conda/lib/python3.9/site-packages/uproot/reading.py", line 141, in open
    file = ReadOnlyFile(
  File "/opt/conda/lib/python3.9/site-packages/uproot/reading.py", line 580, in __init__
    self._source = Source(
  File "/opt/conda/lib/python3.9/site-packages/uproot/source/xrootd.py", line 272, in _

OSError: XRootD error: [FATAL] Auth failed: No protocols left to try
in file root://xcache//store/group/lpcmetx/SIDM/ffNtupleV4/2018/SIDM_XXTo2ATo2Mu2E_mXX-800_mA-0p25_ctau-1p2_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18DRPremix-102X_upgrade2018_realistic_v15-v1/210326_161534/0000/ffNtuple_3.root

In [ ]:
legend_entries = [s[13:] for s in samples]
nplots = 2

# LJs:
plt.subplot(2,1,1) # rows, columns, which subplot
for S in samples:  
    output["out"][S]["hists"]["matched_muon_n"][ch1,:].plot(density=False, flow='none')
plt.title("number of matched muons")
plt.legend(legend_entries, alignment="left")
#-----

# Muons:
plt.subplots(1, nplots, figsize=(nplots*12, 10))
plt.subplot(1,nplots,1) # rows, columns, which subplot
for S in samples:  # genAs_toMu_matched_lj
    output["out"][S]["hists"]["genAs_toMu_invmass"][ch1,:].plot(density=False, flow='none')
plt.title("invariant mass of muons from genAs")
plt.legend(samples)

plt.subplot(1,nplots,2)
for S in samples: # mu_mu_invmass_lowRange
    output["out"][S]["hists"]["muons_matched_muLj_invmass"][ch1,:].plot(density=False, flow='none')
plt.title("invariant mass of matched reco muons")
plt.legend(samples)
#-----

# Electrons:
plt.subplots(1, nplots, figsize=(nplots*12, 10))
plt.subplot(1,nplots,1)
for S in samples: # genAs_toE_matched_lj
    output["out"][S]["hists"]["genAs_toE_invmass"][ch1,:].plot(density=False, flow='none')
plt.title("invariant mass of electrons from genAs")
plt.legend(samples)

plt.subplot(1,nplots,2)
for S in samples: # egm_egm_invmass_lowRange | electron_invmass
    output["out"][S]["hists"]["electrons_matched_egmLj_invmass"][ch1,:].plot(density=False, flow='none')
plt.title("invariant mass of matched reco electrons")
plt.legend(samples)